# Customer Retention Strategy for Model Fitness Gyms

This project aims to analyze customer retention and churn patterns at Model Fitness branches across the USA. Our objective is to conduct an in-depth examination of churned customers, identify key trends, and understand the primary reasons behind membership cancellations.

The insights from this analysis will guide the development of a comprehensive customer retention strategy designed to enhance member loyalty and address the increasing churn rates, which have emerged as a significant business concern. This strategy will reinforce our commitment to improving member retention and overall customer satisfaction.

In [1]:
# General libraries
import pandas as pd
import numpy as np
import datetime as dt
import scipy.stats as stats

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram

# Sklearn libraries for algorithms, metrics, and clustering
from sklearn.cluster import KMeans
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score
)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
)

# Display settings
pd.set_option("display.max_columns", None)

# Chart styling
sns.set(style='whitegrid')
facecolor = '#eaeaea'
font_color = '#525252'
csfont = {'fontname': 'Liberation Serif'}
hfont = {'fontname': 'LiberationSans-Regular'}